In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torchvision import transforms, datasets, models

In [3]:
class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train, transform):
        self.x_train = x_train
        self.t_train = t_train.astype('int32')
        self.transform = transform

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        data = torch.tensor(self.x_train[idx], dtype=torch.float)
        data = torch.transpose(data,0,2)
        data = torch.transpose(data,1,2)
        label = torch.tensor(self.t_train[idx], dtype=torch.long)
        if self.transform:
            data = self.transform(data)
        return data, label


class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test, transform):
        self.x_test = x_test
        self.transform = transform

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        data = torch.tensor(self.x_test[idx], dtype=torch.float)
        data = torch.transpose(data,0,2)
        data = torch.transpose(data,1,2)
        if self.transform:
            data = self.transform(data)
        return data

In [4]:


# 精度向上ポイント: Data Augmentation手法
transform_train =  transforms.Compose([])
transform_test = transforms.Compose([])

# data augmentationとデータ整形
height = 224
width =224
batch_size = 1
transform_train = transforms.Compose(
    [
         transforms.ToPILImage(),
#          transforms.RandomHorizontalFlip(0.2),
         transforms.Resize((height,width)), # 画像によってサイズが変わってはならない。
#          transforms.CenterCrop(192),
         transforms.ToTensor(),
    ]
)
transform_test = transforms.Compose(
[
    transforms.ToPILImage(),
    transforms.Resize((height,width)),
    transforms.ToTensor()
]
)
x_train = np.load('/root/userspace/public/day2/homework2/data/x_train.npy')
t_train = np.load('/root/userspace/public/day2/homework2/data/t_train.npy')
    
# テストデータ
x_test = np.load('/root/userspace/public/day2/homework2/data/x_test.npy')

trainval_data = train_dataset(x_train, t_train, transform_train)
test_data = test_dataset(x_test,transform_test)
val_ratio = 0.1
val_size = int(len(trainval_data) * val_ratio)
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

# 精度向上ポイント: バッチサイズ
batch_size = 64

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class GAP(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
        return F.adaptive_avg_pool2d(x,(1,1)).squeeze()

def init_weights(m):  # Heの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.0)

In [6]:
model_tuned = models.densenet169(pretrained = True)
model_tuned.avgpool = GAP() # dense layerをGAPに変更

for param in model_tuned.parameters(): # pretrained部分の重みを固定する
    param.requires_grad = False
in_dim = 1664 #入力次元数
out_dim = 5 # 出力クラス数
model_tuned.classifier = nn.Sequential(
    nn.BatchNorm1d(in_dim),
    nn.Linear(in_dim,1000), # classifierの書き換え
    nn.Dropout(0.2,inplace=False),
    nn.ReLU(True),
    nn.BatchNorm1d(1000),
    nn.Linear(1000,512),
    nn.ReLU(True),
    nn.Linear(512,out_dim)
)
# (classifier): Linear(in_features=2208, out_features=1000, bias=True)
model_tuned.classifier.apply(init_weights) # 重み・バイアスの初期化を行う
model_tuned.to(device)

Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/checkpoints/densenet169-b2777c0a.pth


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [7]:
lr = 0.01
optimizer_tuned = optim.Adam(model_tuned.parameters(), lr=lr,weight_decay=1e-4)
# optimizer_tuned = optim.SGD(model_tuned.parameters(), lr=0.01, momentum=0.9,weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer_tuned,step_size=3,gamma=0.5)

loss_function = nn.CrossEntropyLoss()
n_epochs =10
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    model_tuned.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        n_train += t.size()[0]
        model_tuned.zero_grad()  # 勾配の初期化
        
        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)
       
        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算
#         l1_lambda = 0.00001
#         l1_norm = sum(p.abs().sum() for p in model_tuned.parameters())
# 
#         loss = loss + l1_lambda * l1_norm
        
#         optimizer_tuned.zero_grad()  
        loss.backward()  # 誤差の逆伝播

        optimizer_tuned.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする
        
        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())
    scheduler.step()
    model_tuned.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)
        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする
        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())
    print('\n learning rate ',optimizer_tuned.param_groups[0]['lr'])
    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch+1,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))


 learning rate  0.01
EPOCH: 1, Train [Loss: 1.631, Accuracy: 0.737], Valid [Loss: 0.518, Accuracy: 0.827]

 learning rate  0.01
EPOCH: 2, Train [Loss: 0.356, Accuracy: 0.880], Valid [Loss: 0.598, Accuracy: 0.821]

 learning rate  0.005
EPOCH: 3, Train [Loss: 0.287, Accuracy: 0.898], Valid [Loss: 0.624, Accuracy: 0.850]

 learning rate  0.005
EPOCH: 4, Train [Loss: 0.173, Accuracy: 0.941], Valid [Loss: 0.704, Accuracy: 0.876]

 learning rate  0.005
EPOCH: 5, Train [Loss: 0.105, Accuracy: 0.961], Valid [Loss: 0.626, Accuracy: 0.876]

 learning rate  0.0025
EPOCH: 6, Train [Loss: 0.090, Accuracy: 0.972], Valid [Loss: 0.576, Accuracy: 0.870]

 learning rate  0.0025
EPOCH: 7, Train [Loss: 0.078, Accuracy: 0.984], Valid [Loss: 0.617, Accuracy: 0.879]

 learning rate  0.0025
EPOCH: 8, Train [Loss: 0.037, Accuracy: 0.990], Valid [Loss: 0.536, Accuracy: 0.890]

 learning rate  0.00125
EPOCH: 9, Train [Loss: 0.032, Accuracy: 0.989], Valid [Loss: 0.638, Accuracy: 0.876]

 learning rate  0.00125


In [13]:
model_tuned.to(device)
model_tuned.eval()
t_pred = []
for x in dataloader_test:
    x = x.to(device)
    output = model_tuned(x)
    pred = output.argmax(1).tolist()
    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('/root/userspace/submission3_densenet169_bathnor2.csv', header=True, index_label='id')

In [13]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');